### Tr for Single Angle Bolted Through One Leg

Calculate factored tension resistance, $T_r$, of a single angle bolted through one leg.

![Angle bolted through one leg](images/bolted-single-angle.svg)

_Note:_ The figure shows 2 rows of 4 bolts through one leg.  The logic below works for
1 or 2 rows and almost any number of bolts per row.

In [1]:
from Designer import Designer as DesignNotes, SST, show
from newparts import Part

In [2]:
Table6 = {'M16': (28,22),    # CSA S16-14 Table 6  - Minimum edge distances
          '3/4': (32,25),
          'M20': (34,26),
          '7/8': (38,28),
          'M22': (38,28),
          'M24': (42,30),
          '1':   (44,32),
         }

def get_Table6(size,col=1):
    assert col in [1,2], "Must select column 1 or 2 from Table 6."
    try:
        return Table6[size][col-1]
    except KeyError:
        pass
    raise KeyError('Cannot determine min edge distance for bolt size: "{}"'.format(size))

In [3]:
BOLT_DIA = {'M16': 16.,
           '3/4': 25.4*3/4,
           'M20': 20.,
           '7/8': 25.4*7/8,
           'M22': 22.,
           'M24': 24.,
           '1': 25.4,
           }

HOLE_DIA = {'M16': 18.,
           '3/4': 22.,
           'M20': 22.,
           '7/8': 24.,
           'M22': 24.,
           'M24': 26.,
           '1': 27.,
           }

BOLT_Fu = {'A325M': 830.,
          'A490M': 1040.,
          'A325': 825.,
          'A490': 1035.,
          }

In [4]:
#           Leg:   g, g1
UG_Angle = {203: (115,75),    # Usual Gauges for Angles, CISC Handbook 11th ed., p. 6-173
           178: (100,65),
           152: (90,60),
           127: (75,50),
           102: (65,),
           89: (50,),
           76: (45,),
           64: (35,),
           51: (29,),
           44: (25,),
            }

# select largest g2 from 45,50,55,60,65,70,75 > 2.7d and edge dist ok

def get_UG_Angle(leg,nlines=1,size='3/4'):
    leg = int(leg)
    if leg not in UG_Angle:
        raise ValueError('Cannot determine gauges for angle leg of size "{}"'.format(leg))
    
    gs = UG_Angle[leg]
    if nlines == 1:
        return gs[0],0
    if len(gs) == 1:
        raise ValueError('Leg size of "{}" cannot have 2 rows of bolts'.format(leg))
    g1 = gs[1]
    min_edge = get_Table6(size,col=2)
    d = BOLT_DIA[size]
    gmin = 2.7*d                         # CSA S16-14 23.3.1
    for g2 in [80,75,70,65,60,55,50,45]:
        if g2 >= gmin and (leg-g1-g2) >= min_edge:
            return g1,g2
    raise ValueError('Unable to determine workable gauges for angle, leg={}, size={}'.format(leg,size))

In [5]:
Bolt = Part( 'Bolts',
              type = 'A325',
              size = '3/4',
              hole_type = 'punched',  # 'punched' or 'drilled'
              Nrows = 1,              # a row is parallel to the load
              Nlines = 4,             # a line is perpendicular to the load
              pitch = 75.,            # spacing measure paprallel to load (along length of angle)
              end_distance = 30.,     # from end of angle to center of closest hole
              )

Bolt.set( Fu = BOLT_Fu[Bolt.type],
           d = BOLT_DIA[Bolt.size],
           ha = HOLE_DIA[Bolt.size] + (0. if Bolt.hole_type == 'drilled' else 2.),  # CSA S16-14 12.3.2
           )
Bolt.vars()

{'Fu': 825.0,
 'Nlines': 4,
 'Nrows': 1,
 '_doc': 'Bolts',
 'd': 19.049999999999997,
 'end_distance': 30.0,
 'ha': 24.0,
 'hole_type': 'punched',
 'pitch': 75.0,
 'size': '3/4',
 'type': 'A325'}

In [6]:
Dsg = 'L102x76x13'
Ag,d,b,t = SST.section(Dsg,'A,D,B,T')
Angle = Part( 'Angle',
               Fy = 350.,
               Fu = 450.,
               dsg = Dsg,
               Ag = Ag,
               d = d,
               b = b,
               t = t,
               bolted_leg = 'long',   # 'short' or 'long'
               m = 1,    # number of faying surfaces
               threads_intercepted = True)
Angle.vars()

{'Ag': 2100.0,
 'Fu': 450.0,
 'Fy': 350.0,
 '_doc': 'Angle',
 'b': 76.2,
 'bolted_leg': 'long',
 'd': 102.0,
 'dsg': 'L102x76x13',
 'm': 1,
 't': 12.7,
 'threads_intercepted': True}

In [7]:
des = DesignNotes('Tr',units='kN',trace=True)

REQ = des.require   # useful abbreviations
CHK = des.check       
REC = des.record

### Sanity check of input data:

In [8]:
REQ(Bolt.size in BOLT_DIA,'bolt size is not one of available sizes',
    size=Bolt.size, SIZES=BOLT_DIA.keys())
REQ(Bolt.hole_type in ['punched','drilled'], "Value of 'hole_type' must be punched or drilled")
REQ(Angle.threads_intercepted in [False,True],"Value of 'threads_intercepted' must be True or False")
REQ(Angle.m in [1,2], 'Number of faying surfaces must be 1 or 2')
REQ(Angle.bolted_leg in ['short','long'],"Value of 'bolted_leg' must be short or long")
REQ(Bolt.Nrows in [1,2],"Nrows must be 1 or 2")
REQ(Bolt.Nlines >= 2 and Bolt.Nlines <= 10,"Nlines must be in range of 2 to 10")

In [9]:
# data derived from input data:  pitch, end_distance, gauges, etc.

Ag,d,b,t,bleg = Angle['Ag,d,b,t,bolted_leg']
if bleg == 'short':  # bolts go thru leg of size 'd'
    d,b = b,d
    
bolt_diameter,pitch = Bolt['d,pitch']
    
min_end_distance = min_edge_distance = get_Table6(Bolt.size,col=1)  # CSA S16-14 22.3.3
if Bolt.Nlines <= 2:
    min_end_distance = 1.5*bolt_diameter  # CSA S16-01 22.3.4

g1,g2 = get_UG_Angle(d,Bolt.Nrows,Bolt.size)
g3 = edge_distance = d-(g1+g2)
Angle.set( g1=g1, g2=g2, g3=g3 )
show('Ag,d,b,t,g1,g2,g3')

Ag = 2100
d  = 102
b  = 76.2
t  = 12.7
g1 = 65
g2 = 0
g3 = 37


### Check Bolting Details
Failure to meet requirements is not a fatal error. Results are reported in the work record.

In [10]:
# CSA S16-14  22.3.1, 22.3.2, 22.3.3, 22.3.4
max_edge_distance = min(12.*t,150.)
REQ(edge_distance > bolt_diameter/2.,
    'Angle leg of {0} mm does not support {1} lines of bolts.'.format(Bolt.d,Bolt.Nlines))
CHK(pitch >= 2.7*Bolt.d,
    'Pitch greater than minimum','pitch',min_pitch=2.7*bolt_diameter)
CHK(edge_distance >= min_edge_distance,
    'Edge distance greater than minimum','edge_distance,min_edge_distance')
CHK(edge_distance <= max_edge_distance,
    'Edge distance less than maximum','edge_distance,max_edge_distance')
end_distance = Bolt.end_distance
CHK(end_distance >= min_end_distance,
    'End distance greater than minimum','end_distance,min_end_distance')

    Pitch greater than minimum?  OK 
      (pitch=75.0, min_pitch=51.43)
    Edge distance greater than minimum?  OK 
      (edge_distance=37.0, min_edge_distance=32)
    Edge distance less than maximum?  OK 
      (edge_distance=37.0, max_edge_distance=150.0)
    End distance greater than minimum?  NG! *****
      (end_distance=30.0, min_end_distance=32)


False

## Strength Calculations

In [11]:
# CSA S16-14  13.1
phi = 0.90
phiu = 0.75
phib = 0.80 
phibr = 0.80

### Gross section yield:

In [22]:
# CSA S16-14   13.2 (a) (i):
Ag,Fy = Angle['Ag,Fy']
REC(phi*Ag*Fy * 1E-3,'Gross area yield','Ag,Fy');

    Gross area yield: Tr = 661.5 kN
       (Ag=2100, Fy=350.0)


### Net section fracture:

In [13]:
# CSA S16-14   13.2 (a) (iii):
Nrows,Nlines,ha = Bolt['Nrows,Nlines,ha']
Ag,t,Fu = Angle['Ag,t,Fu']
An = Ag - Nrows*ha*t                # CSA S16-14 12.3.1 a)
if Nlines >= 4:                     # CSA S16-14   12.3.3.2 (b)
    Ane = 0.80*An
else:
    Ane = 0.60*An
REC(phiu*Ane*Fu * 1E-3.,'Net area fracture','An,Ane');

    Net area fracture: Tr = 484.7 kN
       (An=1795, Ane=1436)


484.70400000000006

### Block shear failure:
![Block Shear Patterns](images/bolted-single-angle-shear-blocks.svg)

In [16]:
#  CSA S16-14    13.11
Nrows,Nlines,ha,e,s = Bolt['Nrows,Nlines,ha,end_distance,pitch']
d,t,g1,g2,Fy,Fu = Angle['d,t,g1,g2,Fy,Fu']
L = (Nlines-1.)*s

# Case 1 - one shear area, tension failure from furthest bolt to edge
An = (d - g1 - (Nrows-0.5)*ha)*t
Agv = (e+L)*t
Ut = 0.6
Fv = (Fy+Fu)/2.
if Fy > 460:     # CSA S16-14 13.11  (foot note)
    Fv = Fy
REC(phiu*(Ut*An*Fu + 0.6*Agv*Fv) * 1E-3,'Block shear (case 1)','An,Agv,Ut');

    Block shear (case 1): Tr = 647.2 kN
       (An=317.5, Agv=3238, Ut=0.6)


In [19]:
# Case 2 - for 2 or more rows of bolts    CSA S16-14 13.11
if Nrows > 1:
    An = (g2 - ha)*t
    Agv = 2.*(e+L)*t
    Ut = 0.6
    REC(phiu*(Ut*An*Fu + 0.6*Agv*Fv) * 1E-3,'Block shear (case 2)','An,Agv,Ut')

In [21]:
# Case 3 - tearout     CSA S16-14 13.11 
An = 0.
Agv = 2.*(e+L)*t*Nrows
Ut = 0.
REC(phiu*(Ut*An*Fu + 0.6*Agv*Fv) * 1E-3,'Block shear (tearout)','An,Agv,Ut');

    Block shear (tearout): Tr = 1166 kN
       (An=0, Agv=6477, Ut=0)


### Fastener strength, bearing-type connection: bolts in shear

In [26]:
m,threads_in = Angle['m,threads_intercepted']
db,Fub,s,Nrows,Nlines = Bolt['d,Fu,pitch,Nrows,Nlines']

n = Nrows*Nlines
L = (Nlines-1.)*s   # length of connection

# CSA S16-14   13.12.1.2 (c)
multiplier = 1.0
if L >= 760.:
    multiplier *= 0.5/0.6
if threads_in:
    multiplier *= 0.70
Ab = (3.1415926*db*db/4.)
Vr = multiplier*0.6*phib*n*m*Ab*Fub * 1E-3
REC(Vr,'Bolt Shear','multiplier,m,n,Ab,Fub')

    Bolt Shear: Tr = 316.0 kN
       (multiplier=0.7, m=1, n=4, Ab=285.0, Fub=825.0)


316.0334493221597

In [15]:
raise Exception('Converted to Here!')

Exception: Converted to Here!

#### Bolts in bearing:

In [ ]:
# CSA S16-09    13.12.1.2:
Br = 3.*phibr*n*t*db*Fu/1000.
REC('Bolt Bearing',n=n,d=db,t=t,Tr=Br)

#### Combined tearout and bearing:

In [ ]:
# combine tearout of bolts closest to end with bearing of remainder
e = end_distance
Agv = 2.*e*t*Nlines
n = Nlines*(Nrows-1)
Tr = phiu*(0.6*Agv*Fv)/1000.
Br = 3.*phibr*n*t*db*Fu/1000.
REC('Bolt bearing + end tearout',n=n,t=t,d=db,Agv=Agv,Tr=Tr+Br)

### Summary:

In [ ]:
des.show('Fy,Fu,AngleDsg,Ag,d,b,t,Fub,bolt_size,bolt_diameter,hole_type',
         'pitch,end_distance,threads_intercepted,shear_type,bolted_leg,Nlines,Nrows',
         'end_distance,edge_distance,g1,g2')
des.summary()          